In [7]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import pickle

In [8]:
movies_path = os.path.join("ml-25m", "movies.csv")
ratings_path = os.path.join("ml-25m", "ratings.csv")

user_mean_path = os.path.join("ml-25m", "user_mean.csv")
ratings_avg_path = os.path.join("ml-25m", "ratings_avg.csv")
ratings_avg2_path = os.path.join("ml-25m", "ratings_real_avg.csv")

final_path = os.path.join("ml-25m", "final.csv")
cosine_path = os.path.join("ml-25m", "cosine.csv")

pivot_table_path = os.path.join("ml-25m", "pivot_table.sparse")

In [4]:
iteration = 0
movies_ids = np.array([])
users_ids = np.array([])

for chunk in pd.read_csv(ratings_path, chunksize=10000):
    movies_ids = np.append(movies_ids, chunk['movieId'].unique())
    users_ids = np.append(users_ids, chunk['userId'].unique())
    iteration += 1
movies_ids = np.unique(movies_ids) 
users_ids = np.unique(users_ids) 

In [34]:
print("iterations: ", iteration)
print("dif movies are: ", movies_ids.shape)
print("dif users are: ", users_ids.shape)

iterations:  2501
dif movies are:  (59047,)
dif users are:  (162541,)


In [6]:
starting_tm = time.time()
pivot_table = csr_matrix((len(users_ids), len(movies_ids)))
for chunk in pd.read_csv(ratings_path, chunksize=100000):
    loop_tm = time.time()
    rows = [i-1 for i in chunk['userId'].tolist()]
    cols = [np.where(movies_ids == j)[0][0] for j in chunk['movieId']]
    vals = chunk['rating'].tolist()
    pivot_table[rows, cols] = vals
    # print("Took {:.3f}".format(time.time()-loop_tm))
print("Took {:.3f}".format(time.time()-starting_tm))

/home/teomandi/anaconda3/envs/project1/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Took 918.479


In [9]:
pickle.dump(pivot_table, open(pivot_table_path, "wb"))
print("PivoTable was stored")

PivoTable was stored


---

In [11]:
ratings = pd.read_csv(ratings_path)

In [12]:
print("ALL movies are: ", len(ratings['movieId'].unique()))
print("ALL users are: ", len(ratings['userId'].unique()))

dif movies are:  59047
dif users are:  162541


In [42]:
# Get mean from ratings 
starting_tm = time.time() 
iteration = 0
for chunk in pd.read_csv(ratings_path, chunksize=10000):
    user_mean = chunk.groupby(by="userId",as_index=False)['rating'].mean()
    ratings_avg_part = pd.merge(chunk, user_mean, on='userId')
    ratings_avg_part['adg_rating']=ratings_avg_part['rating_x']-ratings_avg_part['rating_y']
    if iteration == 0:
        # rating_avg.to_csv(ratings_avg_path)
        ratings_avg = ratings_avg_part
        # print("starting shape: ", ratings_avg.shape)
    else:
        # rating_avg.to_csv(ratings_avg_path, mode = 'a', header = False)
        # print("adding part: ", ratings_avg_part.shape)
        ratings_avg = pd.concat([ratings_avg, ratings_avg_part])  
        # print("starting shape: ", ratings_avg.shape)
        # break
    iteration += 1
    
print("Took {:.3f}".format(time.time()-starting_tm))

Took 1236.958


In [43]:
ratings_avg.shape

(25000095, 6)

In [15]:
ratings = pd.read_csv(ratings_path)
user_mean = ratings.groupby(by="userId",as_index=False)['rating'].mean()
rating_real_avg = pd.merge(ratings, user_mean, on='userId')
rating_real_avg['adg_rating'] = rating_real_avg['rating_x']-rating_real_avg['rating_y']
rating_real_avg.to_csv(ratings_avg2_path)

,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,1,296,5.0,1147880044,3.814286,1.185714
1,1,306,3.5,1147868817,3.814286,-0.314286
2,1,307,5.0,1147868828,3.814286,1.185714
3,1,665,5.0,1147878820,3.814286,1.185714
4,1,899,3.5,1147868510,3.814286,-0.314286


In [44]:
print(rating_real_avg.shape)
print(rating_real_avg.head())

NameError: name 'rating_real_avg' is not defined

In [ ]:
print(rating_avg[:1000].shape)
rating_avg.head()

In [22]:
final=pd.pivot_table(rating_avg[:1000], values='adg_rating',index='userId',columns='movieId')
final_final =  final.fillna(final.mean(axis=0))

In [23]:
print(final_final.shape)
final_final.head()

(4, 869)


movieId,1,29,32,50,62,110,111,150,151,172,...,168252,168612,171763,175197,176101,176371,179819,182823,187541,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.068647,0.802591,0.802591,1.302591,-3.130435,1.369565,0.302591,0.369565,0.869565,0.302591,...,0.302591,0.302591,-0.197409,-0.197409,-0.197409,0.802591,-0.197409,-0.697409,-0.197409,0.802591
2,-0.130435,0.802591,0.802591,1.302591,-3.130435,1.369565,0.302591,0.369565,0.869565,0.302591,...,0.302591,0.302591,-0.197409,-0.197409,-0.197409,0.802591,-0.197409,-0.697409,-0.197409,0.802591
3,0.302591,0.802591,0.802591,1.302591,-3.130435,1.369565,0.302591,0.369565,0.869565,0.302591,...,0.302591,0.302591,-0.197409,-0.197409,-0.197409,0.802591,-0.197409,-0.697409,-0.197409,0.802591
4,-0.378099,0.802591,0.802591,1.302591,-3.130435,1.369565,0.302591,0.369565,0.869565,0.302591,...,0.302591,0.302591,-0.197409,-0.197409,-0.197409,0.802591,-0.197409,-0.697409,-0.197409,0.802591


In [24]:
cosine = cosine_similarity(final_final)
np.fill_diagonal(cosine, 0 )
similarity_with_movie =pd.DataFrame(cosine,index=final_final.index)
similarity_with_movie.columns=final_final.index
similarity_with_movie.head()




userId,1,2,3,4
userId,,,,
1,0.000000,0.976006,0.976567,0.985163
2,0.976006,0.000000,0.932594,0.954750
3,0.976567,0.932594,0.000000,0.960560
4,0.985163,0.954750,0.960560,0.000000
